制作证件照，按下enter/space键完成抠图并进行背景替换

In [1]:
import cv2
import numpy as np

def changeColor(x):
    pass

img = np.zeros((200, 700, 3), np.uint8)
#读取图像
pic = cv2.imread("./input/test.png")
cv2.namedWindow('pic')
#框选ROI，并给出ROI坐标、长宽
ROIx, ROIy, ROIw, ROIh = cv2.selectROI("pic", pic, False)

#cv2.imshow("pic",pic)#显示原图
#参数及输入量
mask = np.zeros(pic.shape[:2], np.uint8)
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)
rect = (ROIx, ROIy, ROIx + ROIw, ROIy + ROIh)
print(rect)

#grabCut生成前景图
cv2.grabCut(pic, mask, rect, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_RECT)
mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
pic2 = pic * mask2[:, :, np.newaxis]

#滚动条制作
cv2.namedWindow('image')
cv2.createTrackbar("blue", "image", 0, 255, changeColor)
cv2.createTrackbar("green", "image", 0, 255, changeColor)
cv2.createTrackbar("red", "image", 0, 255, changeColor)
while (1):
    cv2.imshow("image", pic2)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break
    r = cv2.getTrackbarPos('red', 'image')
    g = cv2.getTrackbarPos('green', 'image')
    b = cv2.getTrackbarPos('blue', 'image')
    pic2[mask2 == 0] = [b, g, r]

cv2.destroyAllWindows()


(49, 40, 427, 512)


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2561: error: (-27:Null pointer) NULL window: 'image' in function 'cvGetTrackbarPos'


在图像中选择旋转矩形框

In [2]:
import cv2
import numpy as np

# 全局变量
start_point = None
end_point = None
angle = 0
drawing = False
image = None

# 计算旋转矩形的四个顶点
def get_rotated_rect_points(center, size, angle):
    w, h = size
    rect = ((center[0], center[1]), (w, h), angle)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    return box

# 鼠标回调函数
def mouse_callback(event, x, y, flags, param):
    global start_point, end_point, drawing
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)
    
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            end_point = (x, y)
    
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        end_point = (x, y)

# 角度滑动条的回调函数
def update_angle(val):
    global angle
    angle = val

def main():
    global start_point, end_point, angle, image

    # 读取输入图像
    image = cv2.imread("./input/apple.jpg")
    if image is None:
        print("Error: Could not open or find the image.")
        return
    
    # 创建窗口
    cv2.namedWindow("Rotated Rectangle Selector")
    cv2.setMouseCallback("Rotated Rectangle Selector", mouse_callback)
    cv2.createTrackbar("Angle", "Rotated Rectangle Selector", 0, 360, update_angle)

    while True:
        img_copy = image.copy()
        
        if start_point and end_point:
            center = ((start_point[0] + end_point[0]) // 2, (start_point[1] + end_point[1]) // 2)
            size = (abs(end_point[0] - start_point[0]), abs(end_point[1] - start_point[1]))
            points = get_rotated_rect_points(center, size, angle)
            cv2.polylines(img_copy, [points], isClosed=True, color=(0, 255, 0), thickness=2)

        # 显示图像
        cv2.imshow("Rotated Rectangle Selector", img_copy)
        
        key = cv2.waitKey(1) & 0xFF
        if key == 27:  # ESC键退出
            break

    cv2.destroyAllWindows()

main()

图像分割制作透明图像

In [4]:
# 按 'r' 键进行分割

import cv2
import numpy as np
import os

# 全局变量
start_point = None
end_point = None
angle = 0
drawing = False
image = None
mask = None
bg_model = None
fg_model = None
rect = None
iteration = 0
all_results = []  # 修改为存储 (result, mask2) 元组
base_image = None

# 计算旋转矩形的四个顶点
def get_rotated_rect_points(center, size, angle):
    w, h = size
    rect = ((center[0], center[1]), (w, h), angle)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    return box

# 鼠标回调函数
def mouse_callback(event, x, y, flags, param):
    global start_point, end_point, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        start_point = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            end_point = (x, y)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        end_point = (x, y)
        # 计算旋转矩形
        center = ((start_point[0] + end_point[0]) // 2, (start_point[1] + end_point[1]) // 2)
        size = (abs(end_point[0] - start_point[0]), abs(end_point[1] - start_point[1]))
        global rect
        rect = (center, size, angle)

# 角度滑动条的回调函数
def update_angle(val):
    global angle
    angle = val

def process_image():
    global image, mask, bg_model, fg_model, rect, iteration, all_results, base_image

    if rect is None:
        return

    # 初始化掩码和模型
    if mask is None:
        mask = np.zeros(image.shape[:2], np.uint8)
        bg_model = np.zeros((1, 65), np.float64)
        fg_model = np.zeros((1, 65), np.float64)

    # 创建前景和背景模型
    mask[:] = 0
    rect_points = get_rotated_rect_points(rect[0], rect[1], rect[2])
    rect_bounding = cv2.boundingRect(rect_points)  # 转换为传统矩形
    cv2.grabCut(base_image, mask, rect_bounding, bg_model, fg_model, 5, cv2.GC_INIT_WITH_RECT)

    # 处理掩码
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    result = base_image * mask2[:, :, np.newaxis]

    # 更新结果列表，保存当前的 result 和 mask2 的副本
    all_results.append((result.copy(), mask2.copy()))

    # 迭代时更新底图为上一次的结果
    base_image = result.copy()

    # 显示图像
    cv2.imshow("Original Image", image)
    cv2.imshow("Segmented Image", result)

    iteration += 1

def main():
    global start_point, end_point, angle, image, mask, bg_model, fg_model, rect, base_image

    # 读取输入图像
    image = cv2.imread("./input/test.png")
    if image is None:
        print("Error: Could not open or find the image.")
        return

    base_image = image.copy()

    # 创建窗口
    cv2.namedWindow("Rotated Rectangle Selector")
    cv2.setMouseCallback("Rotated Rectangle Selector", mouse_callback)
    cv2.createTrackbar("Angle", "Rotated Rectangle Selector", 0, 360, update_angle)

    while True:
        img_copy = base_image.copy()

        if start_point and end_point:
            center = ((start_point[0] + end_point[0]) // 2, (start_point[1] + end_point[1]) // 2)
            size = (abs(end_point[0] - start_point[0]), abs(end_point[1] - start_point[1]))
            points = get_rotated_rect_points(center, size, angle)
            cv2.polylines(img_copy, [points], isClosed=True, color=(0, 255, 0), thickness=2)

        # 显示图像
        cv2.imshow("Rotated Rectangle Selector", img_copy)

        key = cv2.waitKey(1) & 0xFF
        if key == 13:  # 按 'r' 键进行分割
            process_image()
        elif key == 27:  # ESC键退出
            # 保存所有迭代结果
            if all_results:
                # 创建目录（如果不存在）
                output_dir = "./output/2_4/Rectangular_splitting/"
                os.makedirs(output_dir, exist_ok=True)

                for i, (res, mask_iter) in enumerate(all_results):
                    transparent_res = cv2.merge([
                        res[:, :, 0],
                        res[:, :, 1],
                        res[:, :, 2],
                        (mask_iter * 255).astype('uint8')  # 根据 mask2 创建 alpha 通道
                    ])
                    cv2.imwrite(os.path.join(output_dir, f"iteration_{i}.png"), transparent_res)
                print(f"Saved {len(all_results)} iteration images to '{output_dir}'")
            else:
                print("No segmentation results to save.")
            break
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Saved 2 iteration images to './output/2_4/Rectangular_splitting/'


交互式图像分割

In [1]:
import cv2
import numpy as np

# 定义全局变量
drawing = False
rect_mode = None

# 鼠标回调函数
def draw_circle(event, x, y, flags, param):
    global drawing, rect_mode
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
    elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
        if drawing and rect_mode is not None:
            cv2.circle(mask, (x, y), 10, rect_mode, -1)
            if rect_mode == cv2.GC_FGD:
                cv2.circle(picDraw, (x, y), 15, (0, 255, 0), -1)
            elif rect_mode == cv2.GC_BGD:
                cv2.circle(picDraw, (x, y), 15, (0, 0, 255), -1)
            elif rect_mode == cv2.GC_PR_FGD:
                cv2.circle(picDraw, (x, y), 15, (0, 255, 255), -1)
            elif rect_mode == cv2.GC_PR_BGD:
                cv2.circle(picDraw, (x, y), 15, (255, 0, 255), -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# 读取原图像
pic = cv2.imread("./input/test.png")
picDraw = pic.copy()
# 基础参数
mask = np.zeros(pic.shape[:2], np.uint8)
mask[:] = 2
bgdModel = np.zeros((1, 65), np.float64)
fgdModel = np.zeros((1, 65), np.float64)

# 创建窗口并绑定鼠标回调函数
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_circle)

while True:
    picDraw = pic.copy()
    mask = np.zeros(pic.shape[:2], np.uint8)
    mask[:] = 2
    print("Press 'f' for Foreground, 'b' for Background, 'p' for Possible Foreground, 'n' for Possible Background")
    while True:
        cv2.imshow('image', picDraw)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('f'):
            rect_mode = cv2.GC_FGD
        elif key == ord('b'):
            rect_mode = cv2.GC_BGD
        elif key == ord('p'):
            rect_mode = cv2.GC_PR_FGD
        elif key == ord('n'):
            rect_mode = cv2.GC_PR_BGD
        elif key == ord('y'):
            break
        elif key == ord('q'):
            cv2.destroyAllWindows()
            break

    # 应用 GrabCut 算法
    mask, bgdModel, fgdModel = cv2.grabCut(pic, mask, None, bgdModel, fgdModel, 5, cv2.GC_INIT_WITH_MASK)
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
    pic2 = pic * mask2[:, :, np.newaxis]

    cv2.imshow("image", pic2)
    if cv2.waitKey(0) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


Press 'f' for Foreground, 'b' for Background, 'p' for Possible Foreground, 'n' for Possible Background


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\grabcut.cpp:386: error: (-215:Assertion failed) !bgdSamples.empty() && !fgdSamples.empty() in function 'initGMMs'


图像标准工具

In [8]:
import cv2
import numpy as np
import os
import json
import keyboard

class ImageAnnotator:
    def __init__(self, image_list):
        self.image_list = image_list
        self.current_index = 0
        self.drawing = False
        self.start_point = None
        self.end_point = None
        self.rot_rects = []
        self.current_image = None
        self.done_with_current_image = False
        self.current_angle = 0  # 当前角度
        self.temp_rect = None  # 临时存储当前选择框

        self.window_name = 'Image Annotator'
        self.setup_window()
    
    def setup_window(self):
        cv2.namedWindow(self.window_name)
        cv2.setMouseCallback(self.window_name, self.mouse_callback)
        cv2.createTrackbar('Angle', self.window_name, 0, 360, self.update_angle)
        self.load_image()

    def load_image(self):
        if 0 <= self.current_index < len(self.image_list):
            self.current_image = cv2.imread(self.image_list[self.current_index])
            self.rot_rects = []
            self.done_with_current_image = False
            self.current_angle = 0
            self.temp_rect = None
            self.display_image()
        else:
            print("No more images.")
            cv2.destroyAllWindows()
    
    def mouse_callback(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            self.start_point = (x, y)
            self.drawing = True
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.drawing:
                self.end_point = (x, y)
        elif event == cv2.EVENT_LBUTTONUP:
            self.drawing = False
            self.end_point = (x, y)
            self.temp_rect = cv2.minAreaRect(np.array([self.start_point, (self.end_point[0], self.start_point[1]), self.end_point, (self.start_point[0], self.end_point[1])], dtype=np.int32))
            self.display_image()

    def update_angle(self, angle):
        self.current_angle = angle
        self.display_image()

    def display_image(self):
        display_img = self.current_image.copy()
        if self.temp_rect:
            temp_box = cv2.boxPoints((self.temp_rect[0], (self.temp_rect[1][0], self.temp_rect[1][1]), self.current_angle))
            temp_box = np.intp(temp_box)
            cv2.drawContours(display_img, [temp_box], 0, (0, 255, 0), 2)
        for rect in self.rot_rects:
            box = cv2.boxPoints(rect)
            box = np.intp(box)
            cv2.drawContours(display_img, [box], 0, (0, 255, 0), 2)
        cv2.imshow(self.window_name, display_img)

    def add_rot_rect(self):
        if self.temp_rect:
            rect = (self.temp_rect[0], (self.temp_rect[1][0], self.temp_rect[1][1]), self.current_angle)
            self.rot_rects.append(rect)
            self.temp_rect = None

    def save_annotations(self):
        annotations = []
        h, w, _ = self.current_image.shape
        for rect in self.rot_rects:
            box = cv2.boxPoints(rect)
            box = np.array(box)
            x_center = np.mean(box[:, 0]) / w
            y_center = np.mean(box[:, 1]) / h
            width = np.linalg.norm(box[0] - box[1]) / w
            height = np.linalg.norm(box[1] - box[2]) / h
            angle = rect[2]
            annotations.append({
                'center_x': x_center,
                'center_y': y_center,
                'width': width,
                'height': height,
                'angle': angle
            })
        return annotations

    # def handle_key(self, key):
    #     if key == 13:  # Enter key
    #         if self.temp_rect:
    #             self.add_rot_rect()
    #         if not self.done_with_current_image:
    #             self.done_with_current_image = True
    #             print("Press Shift + Enter to confirm and move to the next image.")
    #     elif key == 27:  # ESC key
    #         cv2.destroyAllWindows()
    #         exit()

    def run(self):
        while True:
            key = cv2.waitKey(10) & 0xFF
            
            if key == 27:  # ESC key
                cv2.destroyAllWindows()
                exit()
            elif keyboard.is_pressed('shift') and key == 13:  # Shift + Enter key
                self.save_current_annotations()
                self.current_index += 1
                self.load_image()
            elif key == 13:  # Enter key
                if self.temp_rect:
                    self.add_rot_rect()
                if not self.done_with_current_image:
                    self.done_with_current_image = True
                    print("Press Shift + Enter to confirm and move to the next image.")


    def save_current_annotations(self):
        save_dir = './output/2_4/Image_standard_tools'
        os.makedirs(save_dir, exist_ok=True)
        
        annotations = self.save_annotations()
        save_path = os.path.join(save_dir,f'annotations_{self.current_index}.json')
        with open(save_path, 'w') as f:
            json.dump(annotations, f, indent=4)

def list_images_in_directory(directory):
    supported_formats = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff')
    image_list = [os.path.join(directory, f) for f in os.listdir(directory) if f.lower().endswith(supported_formats)]
    return sorted(image_list)

if __name__ == '__main__':
    # Replace with your directory containing images
    directory_path = './input'
    image_paths = list_images_in_directory(directory_path)
    annotator = ImageAnnotator(image_paths)
    annotator.run()


Press Shift + Enter to confirm and move to the next image.
Press Shift + Enter to confirm and move to the next image.
Press Shift + Enter to confirm and move to the next image.
Press Shift + Enter to confirm and move to the next image.
No more images.


KeyboardInterrupt: 